# `01-addresses-enrich` - Adding Addresses from OpenAddresses

![](../img/data_model_addresses.png)

# TODOs

- [x] update data model image
- [ ] add address nodes using apoc.load.json
- [ ] find nearest intersection to connect address node
- [ ] demonstrate routing with apoc.algo.djikstra and apoc.algo.aStar

In [3]:
# format? maybe just document apoc.load.json cypher and then add ability to run them?

In [5]:
# to use apoc.load.json? or just cypher and pass rows?

```Cypher

CREATE CONSTRAINT FOR (a:Address) REQUIRE a.id IS UNIQUE
```

Create `Address` nodes for each feature in geojson file:

```Cypher
CALL apoc.load.json("file:///sanmateo_addresses.geojson") YIELD value
MERGE (a:Address {id: value.properties.id})
SET a.location = 
  point(
      {latitude: value.geometry.coordinates[1], longitude: value.geometry.coordinates[0]}),
    a.full_address = value.properties.number + " " + value.properties.street + " " + value.properties.city + ", CA " + value.properties.postcode

SET a += value.properties
```

Find nearest intersection for each `Address` node and connect with `NEAREST_INTERSECTION` relationship:

```Cypher
MATCH (p:Address) WHERE NOT EXISTS ((p)-[:NEAREST_INTERSECTION]->(:Intersection))
WITH p LIMIT 100000
CALL {
  WITH p
  MATCH (i:Intersection)
  USING INDEX i:Intersection(location)
  WHERE point.distance(i.location, p.location) < 2000
  
  WITH i
  ORDER BY point.distance(p.location, i.location) ASC 
  LIMIT 1
  RETURN i
}
WITH p, i

MERGE (p)-[r:NEAREST_INTERSECTION]->(i)
SET r.length = point.distance(p.location, i.location)
RETURN COUNT(p)
```

Route between an address and point of interest:

```Cypher
MATCH (a:Address)-[:NEAREST_INTERSECTION]->(source:Intersection)
WHERE a.full_address CONTAINS "410 E 5TH AVE SAN MATEO, CA"
MATCH 
  (poi:PointOfInterest)-[:NEAREST_INTERSECTION]->(dest:Intersection) 
WHERE poi.geometry_id = "w914450392@1"
CALL apoc.algo.dijkstra(source, dest, "ROAD_SEGMENT", "length") 
YIELD weight, path
WITH [ x in nodes(path) | {latitude: x.location.latitude, longitude: x.location.longitude}] AS route, weight AS totalDist
RETURN *
```

![](../img/address_routing.png)